# End to end Deep learning project using simply RNN

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [2]:
## load the imdb dataset
max_features = 10000 # vocabulary size
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 [==============================] - 4s 0us/step


In [3]:
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (25000,), y_train shape: (25000,)
X_test shape: (25000,), y_test shape: (25000,)


In [6]:
# Inspect a sample review and its labels
sample_review = X_train[0]
sample_label = y_train[0]

print(f"sample review (in numbers): {sample_review}")
print(f"sample label: {sample_label}")

sample review (in numbers): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
sample label: 1


In [9]:
## Mapping of word indices back to word (for understanding)
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}
reverse_word_index

{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling',
 52009: "hold's",
 11307: 'comically',
 40830: 'localized',
 30568: 'disobeying',
 52010: "'royale",
 40831: "harpo's",
 52011: 'canet',
 19313: 'aileen',
 52012: 'acurately',
 52013: "diplomat's",
 25242: 'rickman',
 6746: 'arranged',
 52014: 'rumbustious',
 52015: 'familiarness',
 52016: "spider'",
 68804: 'hahahah',
 52017: "wood'",
 40833: 'transvestism',
 34702: "hangin'",
 2338: 'bringing',
 40834: 'seamier',
 34703: 'wooded',
 52018: 'bravora',
 16817: 'grueling',
 1636: 'wooden',
 16818: 'wednesday',
 52019: "'prix",
 34704: 'altagracia',
 52020: 'circuitry',
 11585: 'crotch',
 57766: 'busybody',
 52021: "tart'n'tangy",
 14129: 'burgade',
 52023: 'thrace',
 11038: "tom's",
 52025: 'snuggles',
 29114: 'francesco',
 52027: 'complainers',
 52125: 'templarios',
 40835: '272',
 52028: '273',
 52130: 'zaniacs',

In [10]:
decoded_review = ' '.join([reverse_word_index.get(i-3, '?') for i in reverse_word_index])
decoded_review

"footwork geysers cronyn flourish 'librarian' suppose kiera walter sequels hardboiled tsukino permanently gnawing nationalities nunnery dogtown trawling fiber hold's 'royale amores moron canet acurately diplomat's rediscoveries rumbustious localized brashness voices harpo's matsumoto's familiarness snatchers news 35mm spider' fawn wood' nominee thundercleese bravora leaf circuitry motor pantheistic amenabar gaskets recollection's boop's snuggles anniversaries wooded hypnotism transvestism flemming complainers contributions staying spots 273 altagracia '00s uality seamier ph yomada's gunther warmongering usenet snuggled pawed 'pleasure jaffe sonja 'reloaded' dependency dependence rocque videodrome designing need hopeless branagh's registration matrix' mailings tom'' headlight madhavi jessie kazakos' babaganoosh error' neurologist noe's 270 fudd informative sters quart wells' roars eggar's cleaned etherything cervi's 'reboot' breaker jaekel lemmings gateshead sidebars sforza's murdock mu

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 500

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

# Train the simply RNN

In [17]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length = max_len)) # Embedding layers
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1313025 (5.01 MB)
Trainable params: 1313025 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [25]:
# create an instance of early stopping using callback
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
early_stopping


In [26]:
history = model.fit(
    X_train, y_train, epochs = 10, batch_size = 32,
    validation_split = 0.2,
    callbacks = [early_stopping]
)

Epoch 1/10

625/625 [==============================] - 110s 169ms/step - loss: 10.2870 - val_loss: 0.5953
Epoch 2/10
625/625 [==============================] - 77s 122ms/step - loss: 12941.4863 - val_loss: 0.4795
Epoch 3/10
625/625 [==============================] - 79s 126ms/step - loss: 0.3639 - val_loss: 0.3857
Epoch 4/10
625/625 [==============================] - 76s 122ms/step - loss: 0.4667 - val_loss: 0.4343
Epoch 5/10
625/625 [==============================] - 76s 122ms/step - loss: 0.2433 - val_loss: 0.3911
Epoch 6/10
625/625 [==============================] - 84s 135ms/step - loss: 0.1735 - val_loss: 0.4161
Epoch 7/10
625/625 [==============================] - 83s 132ms/step - loss: 0.1167 - val_loss: 0.4297
Epoch 8/10
625/625 [==============================] - 77s 123ms/step - loss: 0.0921 - val_loss: 0.4576


In [27]:
# Save the model file
model.save("simple_rnn_imdb.h5")

c:\Users\LENOVO\Desktop\Data science projects\rnn_project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
